In [5]:
import numpy as np
from hwcomponents import DataStructures as ds 
from hwcomponents import Utils

In [2]:
# Test Neuron datastructure
ds.Neuron(
    param_leak_str=1,
    param_threshold=20,
    param_reset=0,
    state_core=0
)

Neuron For Memory(leak_str=1, threshold=20, reset=0, core=0)

In [3]:
# Test Synapse datastructure
ds.Synapse(
    length=10,
    word_size=np.uint8
)

Synapse For Memory(length=10, weights=[0 0 0 0 0 0 0 0 0 0])

In [6]:
def random_neuron() -> ds.Neuron:
    return ds.Neuron(
        param_leak_str=(2**3, 2, 0, 2**5),
        param_threshold=(2**6, 2**3, 0, 2**11),
        state_core=0,
        param_reset=(4., 1., 0, 2**3)
    )